**Segmenting and Clustering Neighborhoods in Toronto**

Peer-Graded Assignment 

***Table of Contents***

***Q1***:
1.1 Create Notebook
1.2 Web Scraping
1.3 Data to Pandas df
1.4 Dataframe cleaning 
1.5 Notebook to Github repository 

***Q2***:
2.1 Geocoder for neighborhood coordinates
2.2 Creating request df
2.3 Notebook to Github repository

***Q3***:
3.1 Test set of Toronto boroughs 
3.2 Analysis of Toronto neighborhoods
3.3 Foursquare API for Toronto neighborhoods
3.4 Common venue catagories,Folium maps to visualize neighborhoods. 
3.5 Grouping neighborhood clusters 
3.6 Notebook to Github repository 

***Part Q1 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++***

***1.1 Creat Notebook***


In [8]:
import numpy as np
import pandas as pd 
import requests 

print('Imported')

Imported


***1.2 Web Scraping***

In [9]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('Beautifulsoup and csv imported')

Beautifulsoup and csv imported


In [10]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored')

SSL certificate errors ignored


In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print(soup.prettify())
print('soup ready')

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptG7wpAMNAAAUaw2@sAAAES","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [12]:
table = soup.find('table', {'class': 'wikitable sortable'})


In [13]:
table_rows = table.find_all('tr')

In [14]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()] #filters out bad rows
    
print(df.head(5))
print('***')
print(df.tail(5))

  PostalCode           Borough                Neighborhood
1        M1A      Not assigned                            
2        M2A      Not assigned                            
3        M3A        North York                   Parkwoods
4        M4A        North York            Victoria Village
5        M5A  Downtown Toronto  Regent Park / Harbourfront
***
    PostalCode       Borough  \
176        M5Z  Not assigned   
177        M6Z  Not assigned   
178        M7Z  Not assigned   
179        M8Z     Etobicoke   
180        M9Z  Not assigned   

                                          Neighborhood  
176                                                     
177                                                     
178                                                     
179  Mimico NW / The Queensway West / South of Bloo...  
180                                                     


***1.3 Data to pandas df***

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [16]:
df.shape

(180, 3)

***1.4 Dataframe Cleaning***

In [17]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text, 'lxml')

table = soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]

#df.head(15)

In [18]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#df.head()

In [19]:
df1 = df.reset_index()

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index           103 non-null int64
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [21]:
df1.shape

(103, 4)

In [22]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

df2.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [24]:
df2.shape

(103, 2)

In [25]:
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']

df2.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [26]:
df3 = df2.reset_index()

df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


***Removing duplicate boroughts***

In [27]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")


In [28]:
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [30]:
df3.shape

(103, 3)

***Part Q2 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++***

In [31]:
!pip install geopy

In [32]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
city = "London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlogtitude is:-"
     , loc.longitude)

latitude is :- 51.5073219 
logtitude is:- -0.1276474


In [33]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [34]:
import pandas as pd 
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [35]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighborhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")

In [36]:
df_geopy1 = df3
#df_geopy1

In [37]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighborhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

,PostalCode,Borough,Neighborhood,address
0,M1B,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [38]:
df_geopy1 = df3

In [39]:
df_geopy1.shape

(103, 4)

In [40]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
address         103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [41]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
address         103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [42]:
df_geopy1.head()

,PostalCode,Borough,Neighborhood,address
0,M1B,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [43]:
df_geopy1.shape

(103, 4)

In [44]:
df_geopy1.to_csv('geopy1.csv')

In [45]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode("M1G, Scarborough, Woburn")


#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

In [46]:
!pip install geocoder

In [47]:
df3.to_csv('geopy.csv')

In [48]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['PostalCode'],row['Borough'], row['Neighborhood'] )

In [49]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

In [50]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode("Toronto, Highland Creek")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1C Scarborough Highland Creek,Rouge Hill,Port Union = no address

In [51]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode("Toronto, Morningside")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1E Scarborough Guildwood,Morningside,West Hill = no address

***Get Coordinates***

In [52]:
import pandas, os
#os.listdir()

In [53]:
df_geopy=df3
#df_geopy.head()

In [54]:
import geopy
#dir(geopy)

In [55]:
type(df_geopy)

pandas.core.frame.DataFrame

In [56]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
address         103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


***Importing Geopy***:

In [57]:
!pip install geopy

In [58]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


***Connection to OpenStreetMap***

In [59]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighborhood']
df_geopy.head()

,PostalCode,Borough,Neighborhood,address
0,M1B,Scarborough,Malvern / Rouge,"M1B,Scarborough,Malvern / Rouge"
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C,Scarborough,Rouge Hill / Port Union / High..."
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E,Scarborough,Guildwood / Morningside / West..."
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae"


In [60]:
nom = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")

In [61]:
n=nom.geocode('M1B, Scarborough, Rouge,Malvern')
n

In [62]:
n.latitude

AttributeError: 'NoneType' object has no attribute 'latitude'

In [63]:
type(n)

NoneType

***Watch for None values***

In [64]:
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n2)

None


In [65]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighborhood,address,Coordinates
0,M1B,Scarborough,Malvern / Rouge,"M1B,Scarborough,Malvern / Rouge",None
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C,Scarborough,Rouge Hill / Port Union / High...",None
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E,Scarborough,Guildwood / Morningside / West...",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


In [66]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
address         103 non-null object
Coordinates     2 non-null object
dtypes: object(5)
memory usage: 4.8+ KB


In [67]:
df_geopy.Coordinates[0]

In [68]:
print(df_geopy.Coordinates[1])

None


In [69]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighborhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,Malvern / Rouge,"M1B,Scarborough,Malvern / Rouge",None,NaN,NaN
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C,Scarborough,Rouge Hill / Port Union / High...",None,NaN,NaN
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E,Scarborough,Guildwood / Morningside / West...",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN


In [70]:
df_geopy.to_csv('geo_loc_py.csv')

In [72]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

The latitude of M1B,Scarborough,Malvern / Rouge is nan and its longitude is nan


***2.2 csv to request dataframe***

In [73]:

import pandas as pd 
import requests
import io
 
data2 = pd.read_csv("geopy.csv") 

data2.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,address
0,0,M1B,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,2,M1E,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [74]:

data3 = pd.read_csv("https://raw.githubusercontent.com/Juankboards/toronto_neighborhood_clustering/master/Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
data3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
data3.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#data3.head()

In [76]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighborhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,Malvern / Rouge,"M1B, Scarborough, Malvern / Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,Rouge Hill / Port Union / Highland Creek,"M1C, Scarborough, Rouge Hill / Port Union / Hi..."
2,M1E,43.763573,-79.188711,2,Scarborough,Guildwood / Morningside / West Hill,"M1E, Scarborough, Guildwood / Morningside / We..."
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [77]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
PostalCode      103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
Unnamed: 0      103 non-null int64
Borough         103 non-null object
Neighborhood    103 non-null object
address         103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


In [78]:
cols = data1.columns.tolist()
cols

['PostalCode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighborhood',
 'address']

In [79]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighborhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [80]:
data1 = data1[new_column_order]
data1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [81]:
sorted_df = data1.sort_values([ 'Neighborhood', 'Latitude'], ascending=[True, True])
sorted_df.head()
# no idea how to get it exacly like the exqample

,PostalCode,Borough,Neighborhood,Latitude,Longitude
12,M1S,Scarborough,Agincourt,43.794200,-79.262029
89,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
28,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
19,M2K,NorthYork,Bayview Village,43.786947,-79.385975
62,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [82]:
sorted_df.reset_index(inplace=True)
sorted_df.head()


,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,12,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,89,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,28,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,19,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,62,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [83]:
sorted_cols =sorted_df.columns.tolist()
sorted_cols

['index', 'PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [84]:
new_column_order2 = ['PostalCode',
 'Borough',
 'Neighborhood',
 'Latitude',
 'Longitude']
new_column_order2

['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [85]:
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


***2.3 Notebook to Github repository***

In [86]:
sorted_dataframe.to_csv('sorted_geoloc.csv')

*** Part Q3 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ***

***3.1 Test Boroughs in Toronto***

In [87]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [88]:
# library to handle JSON files

import pandas as pd

import json

sorted_dataframe.to_json(path_or_buf='geo_toronto.json', orient='table')

In [89]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [90]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]
#Let's take a look at the first item in this list.

{'index': 0,
 'PostalCode': 'M1S',
 'Borough': 'Scarborough',
 'Neighborhood': 'Agincourt',
 'Latitude': 43.7942003,
 'Longitude': -79.2620294}

In [91]:
sorted_dataframe.info()
sorted_dataframe.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

In [92]:
sorted_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [93]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


*** How to get coordonates***

In [94]:
address = 'Adelaide'

geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Adelaide are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Adelaide are -34.9281805, 138.5999312.


***How to map with Folium***

In [95]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



***Generate Maps, open them on browser!***

In [96]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


***Map generated with folium default markers***

In [97]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_dataframe.sample(n=11)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)


    
#map_test

#open map_test.html in browser
map_test.save("map_test.html")

# if you cannot generate the maps open PGA_map_*.html from the zip file


***Test Borough data, map with markerClusters:***

In [98]:
from folium.plugins import MarkerCluster
map_borough = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))
    map_borough.add_child(mc)


#map_borough

#open in map_borough.html browser 
map_borough.save("map_borough.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

***3.2 Analysis on Toronto Neighborhoods***

In [99]:
import pandas as pd
import folium



#grab a random sample from df
toronto_n = sorted_dataframe.sample(n=20)
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighborhood))

    
map_toronto 

#open map_toronto.html in browser

map_toronto.save("map_toronto20.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

***3.3 Foursquare API to Explore Neighborhoods in Toronto***

In [100]:
sorted_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M3H,NorthYork,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,Bedford Park / Lawrence Manor East,43.733283,-79.419750


In [101]:
sorted_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [102]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [103]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


***Create Map of Toronto***

In [104]:
# create map of Toronto using latitude and longitude values
map_toronto_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(sorted_dataframe['Latitude'], sorted_dataframe['Longitude'], sorted_dataframe['Borough'], sorted_dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighborhoods)  
    
map_toronto_neighborhoods

map_toronto_neighborhoods.save("map_toronto_neighbourhoods.html")

#open map_toronto_neighborhoods.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [105]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="Clustering_Neighborhoods_in_Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of York, Toronto are 43.67910515, -79.49118414007154.


In [106]:
york_data = sorted_dataframe[sorted_dataframe['Borough'] == 'York'].reset_index(drop=True)
york_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6M,York,Del Ray / Mount Dennis / Keelsdale and Silvert...,43.691116,-79.476013
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
3,M6N,York,Runnymede / The Junction North,43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [107]:
york_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
PostalCode      5 non-null object
Borough         5 non-null object
Neighborhood    5 non-null object
Latitude        5 non-null float64
Longitude       5 non-null float64
dtypes: float64(2), object(3)
memory usage: 280.0+ bytes


In [108]:
# create map of Manhattan using latitude and longitude values
map_york_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york_toronto)  
    
map_york_toronto

map_york_toronto.save("map_york_toronto.html")

#open map_york_toronto.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file


In [109]:
CLIENT_ID = '1N5Y02JWCUQPQPTM01N3UTLYMIGXTXX4PWQQWSBOHL0B3N1K' # your Foursquare ID
CLIENT_SECRET = 'WBWV1QH453VRI2E1DM52CYCYZQFXFWB0UHDF3N1BJ4CIDM1P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('My credentails: ***')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails: ***
CLIENT_ID: 1N5Y02JWCUQPQPTM01N3UTLYMIGXTXX4PWQQWSBOHL0B3N1K
CLIENT_SECRET:WBWV1QH453VRI2E1DM52CYCYZQFXFWB0UHDF3N1BJ4CIDM1P


In [110]:
york_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
PostalCode      5 non-null object
Borough         5 non-null object
Neighborhood    5 non-null object
Latitude        5 non-null float64
Longitude       5 non-null float64
dtypes: float64(2), object(3)
memory usage: 280.0+ bytes


In [111]:
neighborhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = york_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Caledonia-Fairbanks are 43.6890256, -79.453512.


In [112]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 

CLIENT_SECRET, 

VERSION, 

neighborhood_latitude, 

neighborhood_longitude, 

radius, 

LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=1N5Y02JWCUQPQPTM01N3UTLYMIGXTXX4PWQQWSBOHL0B3N1K&client_secret=WBWV1QH453VRI2E1DM52CYCYZQFXFWB0UHDF3N1BJ4CIDM1P&v=20180605&ll=43.6890256,-79.453512&radius=500&limit=100'

In [113]:
york_results = requests.get(url).json()
#york_results

***3.4 Get Common Venue Categories in Neighborhood***

In [114]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [115]:
york_venues = york_results['response']['groups'][0]['items']
    
york_nearby_venues = json_normalize(york_venues) # flatten JSON

# filter columns
york_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
york_nearby_venues = york_nearby_venues.loc[:, york_filtered_columns]

# filter the category for each row
york_nearby_venues['venue.categories'] = york_nearby_venues.apply(get_category_type, axis=1)

# clean columns
york_nearby_venues.columns = [col.split(".")[-1] for col in york_nearby_venues.columns]

york_nearby_venues.head()

,name,categories,lat,lng
0,Nairn Park,Park,43.690654,-79.456300
1,Maximum Woman,Women's Store,43.690651,-79.456333
2,Fairbanks Pool,Pool,43.691959,-79.448922
3,Fairbank Memorial Park,Park,43.692028,-79.448924


In [116]:
york_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
name          4 non-null object
categories    4 non-null object
lat           4 non-null float64
lng           4 non-null float64
dtypes: float64(2), object(2)
memory usage: 208.0+ bytes


In [117]:
print('{} venues were returned by Foursquare.'.format(york_nearby_venues.shape[0]))

4 venues were returned by Foursquare.


***Explore Neighborhoods in York***

In [118]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:
york_venues = getNearbyVenues(names=york_data['Neighborhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Caledonia-Fairbanks
Del Ray / Mount Dennis / Keelsdale and Silverthorn
Humewood-Cedarvale
Runnymede / The Junction North
Weston


In [120]:
york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
1,Caledonia-Fairbanks,43.689026,-79.453512,Maximum Woman,43.690651,-79.456333,Women's Store
2,Caledonia-Fairbanks,43.689026,-79.453512,Fairbanks Pool,43.691959,-79.448922,Pool
3,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park
4,Del Ray / Mount Dennis / Keelsdale and Silvert...,43.691116,-79.476013,Subway,43.690218,-79.474050,Sandwich Place


In [121]:
york_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
Neighbourhood              19 non-null object
Neighbourhood Latitude     19 non-null float64
Neighbourhood Longitude    19 non-null float64
Venue                      19 non-null object
Venue Latitude             19 non-null float64
Venue Longitude            19 non-null float64
Venue Category             19 non-null object
dtypes: float64(4), object(3)
memory usage: 1.1+ KB


In [122]:
york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Caledonia-Fairbanks,4,4,4,4,4,4
Del Ray / Mount Dennis / Keelsdale and Silverthorn,5,5,5,5,5,5
Humewood-Cedarvale,4,4,4,4,4,4
Runnymede / The Junction North,4,4,4,4,4,4
Weston,2,2,2,2,2,2


In [123]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 16 uniques categories.


***3.5 Use to group neighborhoods into clusters***

In [124]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
york_fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[york_fixed_columns]

york_onehot.head()

,Neighbourhood,Brewery,Bus Line,Caribbean Restaurant,Convenience Store,Fast Food Restaurant,Field,Grocery Store,Hockey Arena,Museum,Park,Pizza Place,Pool,Restaurant,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Del Ray / Mount Dennis / Keelsdale and Silvert...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [125]:
york_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 17 columns):
Neighbourhood           19 non-null object
Brewery                 19 non-null uint8
Bus Line                19 non-null uint8
Caribbean Restaurant    19 non-null uint8
Convenience Store       19 non-null uint8
Fast Food Restaurant    19 non-null uint8
Field                   19 non-null uint8
Grocery Store           19 non-null uint8
Hockey Arena            19 non-null uint8
Museum                  19 non-null uint8
Park                    19 non-null uint8
Pizza Place             19 non-null uint8
Pool                    19 non-null uint8
Restaurant              19 non-null uint8
Sandwich Place          19 non-null uint8
Trail                   19 non-null uint8
Women's Store           19 non-null uint8
dtypes: object(1), uint8(16)
memory usage: 536.0+ bytes


In [126]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped.head()

,Neighbourhood,Brewery,Bus Line,Caribbean Restaurant,Convenience Store,Fast Food Restaurant,Field,Grocery Store,Hockey Arena,Museum,Park,Pizza Place,Pool,Restaurant,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.50,0.00,0.25,0.0,0.0,0.00,0.25
1,Del Ray / Mount Dennis / Keelsdale and Silvert...,0.00,0.00,0.2,0.0,0.2,0.00,0.00,0.00,0.2,0.00,0.00,0.00,0.2,0.2,0.00,0.00
2,Humewood-Cedarvale,0.00,0.00,0.0,0.0,0.0,0.25,0.00,0.25,0.0,0.25,0.00,0.00,0.0,0.0,0.25,0.00
3,Runnymede / The Junction North,0.25,0.25,0.0,0.0,0.0,0.00,0.25,0.00,0.0,0.00,0.25,0.00,0.0,0.0,0.00,0.00
4,Weston,0.00,0.00,0.0,0.5,0.0,0.00,0.00,0.00,0.0,0.50,0.00,0.00,0.0,0.0,0.00,0.00


In [127]:
york_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 17 columns):
Neighbourhood           5 non-null object
Brewery                 5 non-null float64
Bus Line                5 non-null float64
Caribbean Restaurant    5 non-null float64
Convenience Store       5 non-null float64
Fast Food Restaurant    5 non-null float64
Field                   5 non-null float64
Grocery Store           5 non-null float64
Hockey Arena            5 non-null float64
Museum                  5 non-null float64
Park                    5 non-null float64
Pizza Place             5 non-null float64
Pool                    5 non-null float64
Restaurant              5 non-null float64
Sandwich Place          5 non-null float64
Trail                   5 non-null float64
Women's Store           5 non-null float64
dtypes: float64(16), object(1)
memory usage: 760.0+ bytes


In [128]:
num_top_venues = 3

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    york_temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    york_temp.columns = ['venue','freq']
    york_temp = york_temp.iloc[1:]
    york_temp['freq'] = york_temp['freq'].astype(float)
    york_temp = york_temp.round({'freq': 2})
    print(york_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1           Pool  0.25
2  Women's Store  0.25


----Del Ray / Mount Dennis / Keelsdale and Silverthorn----
                  venue  freq
0  Caribbean Restaurant   0.2
1  Fast Food Restaurant   0.2
2                Museum   0.2


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2          Park  0.25


----Runnymede / The Junction North----
           venue  freq
0        Brewery  0.25
1       Bus Line  0.25
2  Grocery Store  0.25


----Weston----
               venue  freq
0  Convenience Store   0.5
1               Park   0.5
2            Brewery   0.0




***Put into Pandas df***

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [139]:
num_top_venues = 17

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
york_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

york_neighbourhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue


In [140]:
york_neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

york_neighbourhoods_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,Neighbourhood
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caledonia-Fairbanks
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Del Ray / Mount Dennis / Keelsdale and Silvert...


In [141]:
for ind in np.arange(york_grouped.shape[0]):
    york_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

york_neighbourhoods_venues_sorted.head(2)

ValueError: could not broadcast input array from shape (16) into shape (18)

***Use Folium library to map neighborhoods***

***Run K-means to cluster the neighborhood into 2 cluster***

In [ ]:
# set number of clusters
kclusters = 2

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
york_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
york_kmeans.labels_[0:5]

In [ ]:
# add clustering labels
york_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', york_kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(york_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged

In [ ]:
# create map
york_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(york_map_clusters)
       
york_map_clusters

york_map_clusters.save("york_map_clusters.html")

#open york_map_clusters.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [ ]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

In [ ]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

***3.5 Notebook to Github Repository***